<a href="https://colab.research.google.com/github/gcerbaro/BrainTumor_Detec/blob/main/model_g.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =============================
# 1. Configuração Inicial e Carregamento do Dataset
# =============================
import numpy as np
from tqdm import tqdm
import cv2
import os
import shutil
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, Input, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from google.colab import drive
import glob

# Configurações iniciais
drive.mount('/content/drive')
IMG_PATH = '/content/drive/MyDrive/brain_mri_preprocessed/'
RANDOM_SEED = 123
IMG_SIZE = (224, 224)

# Criar diretórios base
for split in ['TRAIN', 'TEST', 'VAL']:
    for class_name in ['YES', 'NO']:
        os.makedirs(f'{split}/{class_name}', exist_ok=True)

# Dividir os dados
for class_name in os.listdir(IMG_PATH):
    if not class_name.startswith('.'):
        images = os.listdir(os.path.join(IMG_PATH, class_name))
        for n, file_name in enumerate(images):
            src = os.path.join(IMG_PATH, class_name, file_name)
            if n < 5:
                dest = f'TEST/{class_name.upper()}/{file_name}'
            elif n < 0.8 * len(images):
                dest = f'TRAIN/{class_name.upper()}/{file_name}'
            else:
                dest = f'VAL/{class_name.upper()}/{file_name}'
            shutil.copy(src, dest)

Mounted at /content/drive


In [2]:
# =============================
# 2. Funções de Pré-processamento
# =============================
def load_data(dir_path, img_size):
    X, y = [], []
    labels_map = {'NO': 0, 'YES': 1}

    for class_name in ['NO', 'YES']:
        class_path = os.path.join(dir_path, class_name)
        if not os.path.exists(class_path):
            continue

        image_files = glob.glob(os.path.join(class_path, "*.[pj][np]g"))
        for img_path in tqdm(image_files, desc=f"Loading {class_name}"):
            img = cv2.imread(img_path)
            if img is None:
                continue

            # Converter para 3 canais se necessário
            if len(img.shape) == 2:
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
            elif img.shape[2] == 1:
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

            img = cv2.resize(img, img_size)
            X.append(img)
            y.append(labels_map[class_name])

    return np.array(X), np.array(y)

def crop_brain_region(images, img_size):
    cropped_images = []
    for img in tqdm(images, desc="Cropping images"):
        # Converter para escala de cinza
        if img.shape[-1] == 3:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        else:
            gray = img

        # Segmentação do cérebro
        _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            x, y, w, h = cv2.boundingRect(largest_contour)
            cropped_img = img[y:y+h, x:x+w]
            cropped_img = cv2.resize(cropped_img, img_size)
            cropped_images.append(cropped_img)
        else:
            cropped_images.append(cv2.resize(img, img_size))

    return np.array(cropped_images)

def preprocess_mri(img):
    """Pré-processamento otimizado para imagens de MRI cerebral"""
    # Garantir tipo correto
    if img.dtype != np.uint8:
        img = np.clip(img, 0, 255).astype(np.uint8)

    # Converter para escala de cinza
    if img.ndim == 3 and img.shape[2] == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    else:
        gray = img

    # 1. Redução de ruído
    blur = cv2.bilateralFilter(gray, 9, 75, 75)

    # 2. Melhorar contraste
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(blur)

    # 3. Segmentação cerebral
    try:
        _, mask = cv2.threshold(enhanced, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            c = max(contours, key=cv2.contourArea)
            mask_brain = np.zeros_like(mask)
            cv2.drawContours(mask_brain, [c], -1, 255, -1)
            brain_only = cv2.bitwise_and(enhanced, enhanced, mask=mask_brain)
        else:
            brain_only = enhanced
    except:
        brain_only = enhanced

    # 4. Redimensionar e normalizar
    resized = cv2.resize(brain_only, IMG_SIZE)
    normalized = resized.astype(np.float32) / 255.0

    # 5. Converter para 3 canais
    return np.stack([normalized] * 3, axis=-1)

In [3]:
# =============================
# 3. Carregamento e Processamento dos Dados
# =============================
# Carregar dados
X_train, y_train = load_data('TRAIN/', IMG_SIZE)
X_test, y_test = load_data('TEST/', IMG_SIZE)
X_val, y_val = load_data('VAL/', IMG_SIZE)

# Aplicar recorte das regiões cerebrais
X_train_crop = crop_brain_region(X_train, IMG_SIZE)
X_val_crop = crop_brain_region(X_val, IMG_SIZE)
X_test_crop = crop_brain_region(X_test, IMG_SIZE)

# Criar diretórios para imagens processadas
for split in ['TRAIN_CROP', 'TEST_CROP', 'VAL_CROP']:
    for class_name in ['YES', 'NO']:
        os.makedirs(f'{split}/{class_name}', exist_ok=True)

# Salvar imagens processadas
def save_processed_images(images, labels, folder_name):
    for i, (img, label) in enumerate(zip(images, labels)):
        class_dir = 'YES' if label == 1 else 'NO'
        filename = os.path.join(folder_name, class_dir, f"image_{i}.jpg")

        if img.dtype != np.uint8:
            img = (img * 255).astype(np.uint8)
        cv2.imwrite(filename, img)

save_processed_images(X_train_crop, y_train, 'TRAIN_CROP/')
save_processed_images(X_val_crop, y_val, 'VAL_CROP/')
save_processed_images(X_test_crop, y_test, 'TEST_CROP/')

Cropping images: 100%|██████████| 10/10 [00:00<00:00, 1399.45it/s]


In [4]:
# =============================
# 4. Data Augmentation e Modelo
# =============================
# Data generators
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.9, 1.1],
    fill_mode='nearest',
    preprocessing_function=preprocess_mri
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_mri)

BATCH_SIZE = 32
train_generator = train_datagen.flow_from_directory(
    'TRAIN_CROP/',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    seed=RANDOM_SEED
)

validation_generator = val_datagen.flow_from_directory(
    'VAL_CROP/',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    seed=RANDOM_SEED
)

# Modelo otimizado
def create_model():
    base_model = MobileNetV2(
        include_top=False,
        weights='imagenet',
        input_shape=(*IMG_SIZE, 3)
    )
    base_model.trainable = False

    inputs = Input(shape=(*IMG_SIZE, 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs)
    model.compile(
        optimizer=Adam(learning_rate=1e-3),
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )
    return model

model = create_model()

# Callbacks
checkpoint_path = "/content/drive/MyDrive/best_brain_mri_model.h5"
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True, verbose=1),
    ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1)
]

# Treinamento
steps_per_epoch = train_generator.samples // BATCH_SIZE
validation_steps = validation_generator.samples // BATCH_SIZE

print("=== FASE 1: TREINAMENTO BASE ===")
history1 = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=callbacks,
    verbose=1
)

print("=== FASE 2: FINE-TUNING ===")
# Descongelar parcialmente a base
for layer in model.layers[1].layers[-50:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

history2 = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=callbacks,
    verbose=1
)

Found 127 images belonging to 2 classes.
Found 34 images belonging to 2 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
=== FASE 1: TREINAMENTO BASE ===


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5330 - loss: 0.8833 - precision: 0.5523 - recall: 0.6503
Epoch 1: val_accuracy improved from -inf to 0.71875, saving model to /content/drive/MyDrive/best_brain_mri_model.h5


3/3 ━━━━━━━━━━━━━━━━━━━━ 21s 6s/step - accuracy: 0.5261 - loss: 0.8906 - precision: 0.5506 - recall: 0.6054 - val_accuracy: 0.7188 - val_loss: 0.6248 - val_precision: 0.6250 - val_recall: 1.0000 - learning_rate: 0.0010
Epoch 2/20
1/3 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.5312 - loss: 0.7984 - precision: 0.4444 - recall: 0.6154

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_accuracy did not improve from 0.71875
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 776ms/step - accuracy: 0.5312 - loss: 0.7984 - precision: 0.4444 - recall: 0.6154 - val_accuracy: 0.6562 - val_loss: 0.6346 - val_precision: 0.6000 - val_recall: 0.9375 - learning_rate: 0.0010
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3884 - loss: 1.0252 - precision: 0.4042 - recall: 0.5629
Epoch 3: val_accuracy did not improve from 0.71875
3/3 ━━━━━━━━━━━━━━━━━━━━ 21s 7s/step - accuracy: 0.3834 - loss: 1.0224 - precision: 0.3969 - recall: 0.5585 - val_accuracy: 0.6875 - val_loss: 0.6147 - val_precision: 0.8000 - val_recall: 0.5000 - learning_rate: 0.0010
Epoch 4/20
1/3 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.5625 - loss: 0.7226 - precision: 0.7143 - recall: 0.5000
Epoch 4: val_accuracy did not improve from 0.71875
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 924ms/step - accuracy: 0.5625 - loss: 0.7226 - precision: 0.7143 - recall: 0.5000 - val_accuracy: 0.7188 - val_loss: 0.5795 - val_precision: 0

3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - accuracy: 0.5625 - loss: 0.8087 - precision: 0.6000 - recall: 0.5294 - val_accuracy: 0.7500 - val_loss: 0.6300 - val_precision: 0.6818 - val_recall: 0.9375 - learning_rate: 0.0010
Epoch 9/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6010 - loss: 0.7426 - precision: 0.5537 - recall: 0.7586
Epoch 9: val_accuracy did not improve from 0.75000
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.6008 - loss: 0.7378 - precision: 0.5593 - recall: 0.7498 - val_accuracy: 0.6875 - val_loss: 0.6537 - val_precision: 0.6522 - val_recall: 0.8824 - learning_rate: 0.0010
Epoch 10/20
1/3 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.5312 - loss: 0.7209 - precision: 0.5556 - recall: 0.5882
Epoch 10: val_accuracy did not improve from 0.75000

Epoch 10: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.5312 - loss: 0.7209 - precision: 0.5556 - recall: 0.5882 - val_accuracy: 0.7188 - val_loss: 0.64

3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.4194 - loss: 0.7852 - precision: 0.2727 - recall: 0.2308 - val_accuracy: 0.7812 - val_loss: 0.5198 - val_precision: 0.8125 - val_recall: 0.7647 - learning_rate: 1.0000e-04
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.5660 - loss: 0.6989 - precision: 0.7707 - recall: 0.3659
Epoch 5: val_accuracy did not improve from 0.78125
3/3 ━━━━━━━━━━━━━━━━━━━━ 19s 8s/step - accuracy: 0.5755 - loss: 0.6967 - precision: 0.7632 - recall: 0.3724 - val_accuracy: 0.7812 - val_loss: 0.5771 - val_precision: 0.7647 - val_recall: 0.8125 - learning_rate: 1.0000e-04
Epoch 6/10
1/3 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.4516 - loss: 0.6880 - precision: 0.3333 - recall: 0.3077
Epoch 6: val_accuracy did not improve from 0.78125
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.4516 - loss: 0.6880 - precision: 0.3333 - recall: 0.3077 - val_accuracy: 0.7500 - val_loss: 0.6063 - val_precision: 0.7647 - val_recall: 0.7647 - learning_rate: 1.0000

In [5]:
# =============================
# 5. Avaliação Final
# =============================
print("=== AVALIAÇÃO FINAL ===")
validation_generator.reset()
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(
    validation_generator,
    steps=validation_generator.samples // BATCH_SIZE
)

print(f"\n=== RESULTADOS FINAIS ===")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")

# Salvar modelo final
final_model_path = "/content/drive/MyDrive/final_brain_mri_model.h5"
model.save(final_model_path)
print(f"\nModelo salvo em: {final_model_path}")

=== AVALIAÇÃO FINAL ===
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.7812 - loss: 0.5918 - precision: 0.7647 - recall: 0.8125



=== RESULTADOS FINAIS ===
Accuracy: 0.7812
Precision: 0.7647
Recall: 0.8125

Modelo salvo em: /content/drive/MyDrive/final_brain_mri_model.h5
